In [1]:
import cPickle, gzip
import theano.tensor as T
import numpy as np
import theano
from keras.models import Model
from keras.layers import Dense, Input
from keras.utils import np_utils
from sklearn.metrics import accuracy_score

Using Theano backend.


In [2]:
from keras.datasets import mnist
# from keras.datasets import cifar10
import numpy as np

(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [3]:
nb_classes = 10
x_train = x_train.reshape((len(x_train),np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
# x_train = x_train[:,0,0:32,0:32]
# x_test = x_test[:,0,0:32,0:32]
x_train = x_train.astype('float32')/ 255
x_test = x_test.astype('float32')/255
y_train = x_train.astype('int32')
y_test = x_test.astype('int32')


In [4]:
# print np.shape(x_train)
print np.shape(y_test)

(10000, 784)


In [5]:
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()
#%%
train_set_x = train_set[0]
train_set_y = train_set[1]
test_set_x = test_set[0]
test_set_y = test_set[1]
valid_set_x = valid_set[0]
valid_set_y = valid_set[1]

In [8]:
train_set_ind = y2indicator(train_set_y)
test_set_ind = y2indicator(test_set_y)
val_set_ind = y2indicator(valid_set_y)

In [7]:
def y2indicator(y):
    N = len(y)
    ind = np.zeros((N,10))
    for i in xrange(N):
        ind[i,y[i]] = 1
    return ind

def relu(x):
    return x*(x > 0)

def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(a):
    expA = np.exp(a)
    return expA/np.sum(expA,axis=1, keepdims=True)

In [52]:
# train_set_ind = y2indicator(y_train)
# test_set_ind = y2indicator(y_test)
# val_set_ind = y2indicator(valid_set_y)
# print train_set_ind

[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [9]:
def addLayer(inputMat,D,M):
    weight = theano.shared(np.random.randn(D,M))
    print D
    print M
    
#    if addScaling:
#        scale = theano.shared(np.random.randn(D),'scale')
#    else:
#        scale = theano.shared(np.random.randn(D),'scale')
#    actualScale = sigmoid(scale)
#    getScale = theano.function(inputs = [scale], outputs = [actualScale])
#    aScale = getScale(scale)   
#    hiddenLayer = multiplyMatrices(inputMatrix, weight, aScale)
    h = T.dot(weight, inputMat)
    bias = theano.shared(np.random.randn(D,1), broadcastable=(False,True))
#     print T.shape(bias)
    hiddenLayer = relu(h+bias)    
#    return [hiddenLayer,weight,scale]
    return [hiddenLayer, weight,bias]


In [11]:
inputMatrix = T.matrix('inputVector')
targetMatrix = T.matrix('targetMatrix')

M = 784

D = 500

N = 10
[hidden1, W1, b1] = addLayer(inputMatrix,10,M)
# [hidden2, W2, b2] = addLayer(hidden1,N,D)
# [hidden3, W3, b3] = addLayer(hidden2,D,D)
# [hidden4, W4, b4] = addLayer(hidden3,D,D)
# [hidden5, W5, b5] = addLayer(hidden4,D,D)
# [hidden6, W6, b6] = addLayer(hidden5,N,D)
#print hidden1
#print W1
#print b1
outputProbabilities = T.nnet.softmax(hidden1)

getOutputProbs = theano.function(inputs=[inputMatrix],outputs=[outputProbabilities])

10
784


In [12]:
cost = -(targetMatrix*T.log(hidden1)).sum()
prediction = T.argmax(outputProbabilities,axis=0)
accuracy = T.mean(T.neq(prediction, targetMatrix))
lr = 0.02
# op = getOutputProbs
get_prediction = theano.function(inputs = [outputProbabilities],outputs = [prediction])
get_accuracy = theano.function(inputs=[prediction,targetMatrix],outputs=[accuracy])

In [13]:
update_W1 = W1 - lr*T.grad(cost,W1)
# update_W2 = W2 - lr*T.grad(cost,W2)
# update_W3 = W3 - lr*T.grad(cost,W3)
# update_W4 = W4 - lr*T.grad(cost,W4)
# update_W5 = W5 - lr*T.grad(cost,W5)
# update_W6 = W6 - lr*T.grad(cost,W6)

update_b1 = b1 - lr*T.grad(cost,b1)
# update_b2 = b2 - lr*T.grad(cost,b2)
# update_b3 = b3 - lr*T.grad(cost,b3)
# update_b4 = b4 - lr*T.grad(cost,b4)
# update_b5 = b5 - lr*T.grad(cost,b5)
# update_b6 = b6 - lr*T.grad(cost,b6)

In [14]:
# train = theano.function(inputs = [inputMatrix,targetMatrix],
#                         updates = [(W1,update_W1),(W2,update_W2),(W3,update_W3),(W4,update_W4),(W5,update_W5),(W6,update_W6), (b1,update_b1),(b2,update_b2),(b3,update_b3),(b4,update_b4),(b5,update_b5),(b6,update_b6)] ,
#                         )

train = theano.function(inputs = [inputMatrix,targetMatrix],updates = [(W1,update_W1),(b1,update_b1)])


In [15]:
max_iter = 50;
n_batch = 195
batch_sz = 256
train_set_ind = train_set_ind.astype('float32')

In [16]:
for i in xrange(max_iter):
    for j in xrange(n_batch):
        Xbatch = train_set_x[j*batch_sz:(j*batch_sz+batch_sz),]
        Ybatch = train_set_ind[j*batch_sz:(j*batch_sz+batch_sz),]
#         print np.shape(Ybatch)
#         print j
        Ybat = train_set_y[j*batch_sz:(j*batch_sz+batch_sz),]
#         print Ybat
#         print np.shape(Ybat)
#         print np.shape(Xbatch)
#         print np.shape(Ybatch)
        X = np.transpose(Xbatch)
#         print np.shape(X)
        train(X,np.transpose(Ybatch))
#         print j
#         if j% 10 == 0:
    outProb = getOutputProbs(X)
#     print outProb        
    pred_val = get_prediction(outProb[0])
#             print pred_val
#             print np.shape
#             accuracy = get_accuracy(pred_val,Ybat)
#             print accuracy
#             print np.shape(outProb[0])
#             print np.shape(pred_val)
#             print np.shape(np.transpose(pred_val))
#             print np.shape(Ybat)
#     print Xbatch
#     print Ybatch
#     print np.shape(np.transpose(pred_val))
#     print np.shape(Ybat)
    accuracy = accuracy_score(np.transpose(pred_val), Ybat)
#             print cost_val
    print accuracy

0.09765625
0.09765625
0.09765625
0.09375
0.08984375
0.08984375
0.08984375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375
0.0859375


In [ ]:
def addScale(weight,inputMatrix):
        scale = sigmoid(theano.shared(np.random.randn(D),'scale'))
        
        scaledHidden = 2*scale*T.dot(weight,inputMatrix)
        
        return [scale, scaledHidden]

In [ ]:
[S1,SH1] = addScale(W1,inputMatrix)
[S2, SH2] = addScale(W2,SH1)
[S3, SH3] = addScale(W3,SH2)
[S4, SH4] = addScale(W4,SH3)
[S5, SH5] = addScale(W5,SH4)
[S6, SH6] = addScale(W6,SH5)

outputProb = T.nnet.softmax(SH6)

In [ ]:
targetMatTest = T.matrix('test')
inputMatrixTest = T.matrix('inputTest')
costTest = -(targetMatTest*T.log(SH6,axis=1))

lr2 = 0.02

In [ ]:
update_S1 = S1 - lr2*T.grad(cost,S1)
update_S2 = S2 - lr2*T.grad(cost,S2)
update_S3 = S3 - lr2*T.grad(cost,S3)
update_S4 = S4 - lr2*T.grad(cost,S4)
update_S5 = S5 - lr2*T.grad(cost,S5)
update_S6 = S6 - lr2*T.grad(cost,S6)

In [ ]:
testFine = theano.function(
    inputs = [inputMatrixTest,targetMatTest],
    updates = [(S1,update_S1),(S2,update_S2),(S3,update_S3),(S4,update_S4),(S5,update_S5),(S6,update_S6)]
)

In [ ]:
get_prediction_test = theano.function(
    inputs = [inputMatrixTest,targetMatTest],
    outputs = [costTest,outputProb],
)

In [ ]:
max_iter = 10000;
n_batch = 256
batch_sz = 256

In [ ]:
for i in xrange(max_iter):
    for j in xrange(n_batch):
        Xbatch = Xval[j*batch_sz:(j*batch_sz+batch_sz),]
        Ybatch = Yval[j*batch_sz:(j*batch_sz+batch_sz),]
        
        testFine(Xbatch,Ybatch)
        if j% 10 == 0:
            cost_val, pred_val = get_prediction_test(Xval,Yval)
            print cost_val
            print pred_val